In [1]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
from transformers.trainer_callback import TrainerCallback

In [2]:
model_name = "EleutherAI/pythia-70m"
# Get the current directory
current_directory = os.getcwd()
# Join the folder path
folder_path = os.path.join(current_directory, "content")
dataset_name = "ai-medical-chatbot_processed.jsonl"
dataset_path = os.path.join(folder_path, dataset_name)
#dataset_path = f"/content/{dataset_name}"
use_hf = False
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

2024-04-09 18:13:23,020 - DEBUG - utilities - Config: datasets.path: c:\Blog\How-to-Finetuning-Large-Language-Models\content\ai-medical-chatbot_processed.jsonl
datasets.use_hf: false
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize False c:\Blog\How-to-Finetuning-Large-Language-Models\content\ai-medical-chatbot_processed.jsonl


2024-04-09 18:13:23,341 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-f1c6af33428df321/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/dataset_info.json
2024-04-09 18:13:23,429 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-f1c6af33428df321/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/dataset_info.json
2024-04-09 18:13:24,355 - DEBUG - utilities - Select CPU device


In [3]:
def train_model(hyperparameters):
  max_steps = hyperparameters["max_steps"]
  trained_model_name = f"ai_medical_{max_steps}_steps"
  output_dir = trained_model_name
  training_args = TrainingArguments(
    # Learning rate
    learning_rate=hyperparameters["learning_rate"],

    # Number of training epochs
    num_train_epochs=hyperparameters["num_train_epochs"],

    # Max steps to train for (each step is a batch of data)
    # Overrides num_train_epochs, if not -1
    max_steps=max_steps,

    # Batch size for training
    per_device_train_batch_size=hyperparameters["per_device_train_batch_size"],

    # Directory to save model checkpoints
    output_dir=output_dir,

    # Other arguments
    overwrite_output_dir=False, # Overwrite the content of the output directory
    disable_tqdm=False, # Disable progress bars
    eval_steps=120, # Number of update steps between two evaluations
    save_steps=120, # After # steps model is saved
    warmup_steps=1, # Number of warmup steps for learning rate scheduler
    per_device_eval_batch_size=1, # Batch size for evaluation
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    optim=hyperparameters["optim"],
    gradient_accumulation_steps = hyperparameters['gradient_accumulation_steps'],
    gradient_checkpointing=False,
    # Parameters for early stopping
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
  )
  base_model.to(device)
  model_flops = (
    base_model.floating_point_ops(
      {
        "input_ids": torch.zeros(
            (1, training_config["model"]["max_length"])
        )
      }
    )
    * training_args.gradient_accumulation_steps
  )

  #print(base_model)
  print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
  print("Flops", model_flops / 1e9, "GFLOPs")

  trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
  training_output = trainer.train()
  # Evaluate the model
  eval_results = trainer.evaluate()

  return eval_results, training_output


In [4]:
hyperparameters={'learning_rate': 1e-06,
'num_train_epochs': 1,
'per_device_train_batch_size': 1,
'optim': 'adafactor',
'num_iterations': 1,
'max_steps':3,
'gradient_accumulation_steps':2}
#eval_results, training_output =train_model(hyperparameters)
import itertools

In [5]:
from tqdm import tqdm

In [6]:
def find_best_hyperparameters():
        best_hyperparameters = None
        best_loss = float('inf')
        # Define hyperparameter search space
        hyperparameter_space = {
        "learning_rate": [1e-6, 1e-5, 1e-4],
        "num_train_epochs": [1,5,10,20],
        "per_device_train_batch_size": [1],
        "optim": ["adafactor"],
        "num_iterations": [1],
        "max_steps": [3],
        "gradient_accumulation_steps": [3],
        }
        # Generate all combinations of hyperparameters
        all_hyperparameters = list(itertools.product(*hyperparameter_space.values()))

        # Assuming all_hyperparameters is a list of hyperparameter combinations
        for hyperparameter_values in tqdm(all_hyperparameters):
            hyperparameters = dict(zip(hyperparameter_space.keys(), hyperparameter_values))
            
            # Evaluate the model
            # Print the current hyperparameters
            print("Using hyperparameters:")
            for key, value in hyperparameters.items():
                print(f"{key}: {value}")
            eval_results, training_output = train_model(hyperparameters)

            # Check if this set of hyperparameters gives better results
            if eval_results["eval_loss"] < best_loss:
                    best_loss = eval_results["eval_loss"]
                    best_hyperparameters = hyperparameters

        return best_hyperparameters, best_loss

In [7]:
# Call the function to find the best hyperparameters
best_hyperparameters, best_loss = find_best_hyperparameters()

print("Best hyperparameters:", best_hyperparameters)
print("Best loss:", best_loss)

  0%|          | 0/12 [00:00<?, ?it/s]

Using hyperparameters:
learning_rate: 1e-06
num_train_epochs: 1
per_device_train_batch_size: 1
optim: adafactor
num_iterations: 1
max_steps: 3
gradient_accumulation_steps: 3
Memory footprint 0.30687256 GB
Flops 1646.750859264 GFLOPs


  0%|          | 0/3 [00:00<?, ?it/s]

2024-04-09 18:13:26,191 - DEBUG - utilities - Step (1) Logs: {'loss': 5.2655, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}
  0%|          | 0/12 [00:01<?, ?it/s]

{'loss': 5.2655, 'learning_rate': 1e-06, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


...

2024-04-09 18:15:39,688 - DEBUG - utilities - Step (3) Logs: {'loss': 1.7966, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 1.1598482131958008, 'flops': 1419798591340.3328, 'remaining_time': 0.0}
 92%|█████████▏| 11/12 [02:14<00:12, 12.62s/it]

{'loss': 1.7966, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 1.1598482131958008, 'flops': 1419798591340.3328, 'remaining_time': 0.0}
{'train_runtime': 3.4156, 'train_samples_per_second': 2.635, 'train_steps_per_second': 0.878, 'train_loss': 1.4144899249076843, 'epoch': 0.01, 'iter_time': 1.1611294746398926, 'flops': 1418231898535.446, 'remaining_time': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

2024-04-09 18:15:50,110 - DEBUG - utilities - Step (3) Logs: {'eval_loss': 5.092215061187744, 'eval_runtime': 10.4103, 'eval_samples_per_second': 9.606, 'eval_steps_per_second': 9.606, 'epoch': 0.01, 'iter_time': 6.370881915092468, 'flops': 258480832200.46603, 'remaining_time': 0.0}
100%|██████████| 12/12 [02:25<00:00, 12.10s/it]

Best hyperparameters: {'learning_rate': 1e-05, 'num_train_epochs': 20, 'per_device_train_batch_size': 1, 'optim': 'adafactor', 'num_iterations': 1, 'max_steps': 3, 'gradient_accumulation_steps': 3}
Best loss: 3.8763885498046875
